In [46]:
import os
import re
import sys
import copy

ROOT = os.path.join(os.path.dirname(os.getcwd()), 'backend')
if ROOT not in sys.path:
    sys.path.append(ROOT)
    
from moz_sql_parser import parse as ps
sys.setrecursionlimit(1000000)
import json
import os
import pandas as pd
from termcolor import colored
import numpy as np
from sentence_transformers import SentenceTransformer, util
from sklearn.cluster import AgglomerativeClustering
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

In [32]:
from app.dataService.dataService import DataService

In [33]:
# Generate SQL and the execution results from the plain text
text = "films and film prices that cost below 10 dollars"
db_id = "cinema"
dataService = DataService("spider")

=== begin loading model ===
before load_trees
before connecting
before load_trees
before connecting
{'amp': True, 'base_dim': 32, 'batch_size': 20, 'beam_encoder_num_layers': 1, 'cntx_beam': False, 'cntx_rep': False, 'cntx_reranker': True, 'debug': False, 'disentangle_cntx': True, 'embedding_dim': 256, 'grad_acum': 4, 'grad_clip': None, 'grad_norm': None, 'is_oracle': False, 'lin_after_cntx': False, 'lm_lr': 3e-06, 'load_less': False, 'lr': 0.000186, 'max_steps': 60000, 'num_heads': 8, 'power': 0.5, 'rat_dropout': 0.2, 'rat_layers': 8, 'should_rerank': False, 'temperature': 1, 'tfixup': False, 'tiny_dataset': False, 'train_as_dev': False, 'tree_rep_transformer_num_layers': 1, 'uniquify': False, 'use_bce': False, 'use_treelstm': False, 'utt_aug': True, 'value_pred': True}
=== finish loading model ===


In [56]:
# read spider dataset
# dataset_type = "train_spider"
dataset_type="dev"
with open(f"../backend/app/data/dataset/spider/{dataset_type}.json", "r") as f:
    spider_data = json.load(f)
df = pd.DataFrame(spider_data)

In [57]:
if dataset_type == "train_spider":
    df.drop([3153], inplace=True)
df

,db_id,query,query_toks,query_toks_no_value,question,question_toks,sql
0,concert_singer,SELECT count(*) FROM singer,"[SELECT, count, (, *, ), FROM, singer]","[select, count, (, *, ), from, singer]",How many singers do we have?,"[How, many, singers, do, we, have, ?]","{'from': {'table_units': [['table_unit', 1]], ..."
1,concert_singer,SELECT count(*) FROM singer,"[SELECT, count, (, *, ), FROM, singer]","[select, count, (, *, ), from, singer]",What is the total number of singers?,"[What, is, the, total, number, of, singers, ?]","{'from': {'table_units': [['table_unit', 1]], ..."
2,concert_singer,"SELECT name , country , age FROM singer ORDE...","[SELECT, name, ,, country, ,, age, FROM, singe...","[select, name, ,, country, ,, age, from, singe...","Show name, country, age for all singers ordere...","[Show, name, ,, country, ,, age, for, all, sin...","{'from': {'table_units': [['table_unit', 1]], ..."
3,concert_singer,"SELECT name , country , age FROM singer ORDE...","[SELECT, name, ,, country, ,, age, FROM, singe...","[select, name, ,, country, ,, age, from, singe...","What are the names, countries, and ages for ev...","[What, are, the, names, ,, countries, ,, and, ...","{'from': {'table_units': [['table_unit', 1]], ..."
4,concert_singer,"SELECT avg(age) , min(age) , max(age) FROM s...","[SELECT, avg, (, age, ), ,, min, (, age, ), ,,...","[select, avg, (, age, ), ,, min, (, age, ), ,,...","What is the average, minimum, and maximum age ...","[What, is, the, average, ,, minimum, ,, and, m...","{'from': {'table_units': [['table_unit', 1]], ..."
...,...,...,...,...,...,...,...
1029,singer,SELECT Citizenship FROM singer WHERE Birth_Yea...,"[SELECT, Citizenship, FROM, singer, WHERE, Bir...","[select, citizenship, from, singer, where, bir...",What are the citizenships that are shared by s...,"[What, are, the, citizenships, that, are, shar...","{'from': {'table_units': [['table_unit', 0]], ..."
1030,real_estate_properties,SELECT count(*) FROM Other_Available_Features,"[SELECT, count, (, *, ), FROM, Other_Available...","[select, count, (, *, ), from, other_available...",How many available features are there in total?,"[How, many, available, features, are, there, i...","{'from': {'table_units': [['table_unit', 2]], ..."
1031,real_estate_properties,SELECT T2.feature_type_name FROM Other_Availab...,"[SELECT, T2.feature_type_name, FROM, Other_Ava...","[select, t2, ., feature_type_name, from, other...",What is the feature type name of feature AirCon?,"[What, is, the, feature, type, name, of, featu...","{'from': {'table_units': [['table_unit', 2], [..."
1032,real_estate_properties,SELECT T2.property_type_description FROM Prope...,"[SELECT, T2.property_type_description, FROM, P...","[select, t2, ., property_type_description, fro...",Show the property type descriptions of propert...,"[Show, the, property, type, descriptions, of, ...","{'from': {'table_units': [['table_unit', 3], [..."


## sequence data mining & initial labeling

In [58]:
import random

max_seq = 5
min_seq = 2
min_sim_th = 0.2
max_sim_th = 0.75

def cos_sine_sim(sen0, sen1):
    embeddings1 = model.encode(sen0, convert_to_tensor=False)
    embeddings2 = model.encode(sen1, convert_to_tensor=False)
    cosine_scores = util.pytorch_cos_sim(embeddings1, embeddings2)
    return cosine_scores

In [59]:
# generate sequence
import torch
from tqdm import tqdm
# print("randint: ", random.randint(2,4))

all_seqs_num = 0
all_seqs = {}

for name, group in tqdm(df.groupby("db_id")):
    # 1. NL query length
    nl_len = group.question_toks.agg(len)
    # 2. SQL length
    sql_len = group.query_toks_no_value.agg(len)
    group["nl_sql_len"] = (nl_len + sql_len)/2
    # 3. sort by sql_nl length
    group = group.sort_values('nl_sql_len', ascending=True)
    
    if len(group)>=2:
        sql_q = group["query"].values # np.array([" ".join(v) for v in group.query_toks_no_value], dtype="object")
        nl_q = group.question.values
        
        selected = []
        nl_sq = []
        sql_sq = []
        for idx, (nl, sql) in enumerate(zip(nl_q, sql_q)):
            if idx not in selected:
                # natural language/sql similarity
                nl_sim = cos_sine_sim(nl, nl_q).numpy()[0]
                sql_sim  = cos_sine_sim(sql, sql_q).numpy()[0]
                # average similarity
                mean_sim = nl_sim
                # mean_sim = (np.abs(nl_sim) + np.abs(sql_sim))/2
                # print(f"NL sim: {nl_sim} \n SQL sim: {sql_sim} \n Mean sim: {mean_sim}")
                # sort out similarity value
                sort_idx = mean_sim.argsort()[::-1]
                sort_idx_filtered = [sid for sid in sort_idx if (mean_sim[sid] <= max_sim_th) and (mean_sim[sid]>min_sim_th) and (sid not in selected)]
                # print(mean_sim[sort_idx_filtered])
                if len(sort_idx_filtered)>=2:
                    ############ random number for sequence length #################
                    # rand_num = random.randint(min_seq, max_seq)
                    select_num = max_seq if len(sort_idx_filtered)>=max_seq else len(sort_idx_filtered)
                    selected.extend(sort_idx_filtered[:select_num])
                    nl_sq.append(nl_q[sort_idx_filtered[:select_num]].tolist())
                    sql_sq.append(sql_q[sort_idx_filtered[:select_num]].tolist())
        all_seqs_num += len(nl_sq)
        if len(nl_sq)>0:
            all_seqs[name] = {}
            all_seqs[name]["nl"] = nl_sq
            all_seqs[name]["sql"] = sql_sq
print(f"all_seqs_num: {all_seqs_num}")
with open(f"../backend/app/data/dataset/spider/query_seq_1.json", "w") as f:
    json.dump(all_seqs, f)
print(len(all_seqs.keys()), all_seqs.keys())

100%|██████████| 20/20 [00:21<00:00,  1.06s/it]

all_seqs_num: 202
20 dict_keys(['battle_death', 'car_1', 'concert_singer', 'course_teach', 'cre_Doc_Template_Mgt', 'dog_kennels', 'employee_hire_evaluation', 'flight_2', 'museum_visit', 'network_1', 'orchestra', 'pets_1', 'poker_player', 'real_estate_properties', 'singer', 'student_transcripts_tracking', 'tvshow', 'voter_1', 'world_1', 'wta_1'])


In [60]:
with open(f"../backend/app/data/dataset/spider/query_seq_1.json", "r") as f:
    seq_data = json.load(f)

seq_len = 0
for k in seq_data.keys():
    for s in seq_data[k]["nl"]:
        seq_len += len(s)-1
print("pairwise sequence number: {}".format(colored(seq_len, "red")))

pairwise sequence number: 775


## Seq examples

1. select distinct building from faculty 
['In which buildings are there at least ten professors?'
 'Find the building that has the largest number of faculty members.'
 'Show all the distinct buildings that have faculty rooms.'
 'How many faculty members does each building have? List the result with the name of the building.'
 'Show all the buildings along with the number of faculty members the buildings have.'
 'How many faculty do we have?'
 'What is the first and last name of the faculty participating in the most activities?']
 
2. select activity_name from activity ['What activities do we have?' 'How many activities do we have?'
 'Find the number of activities available.']
 
3. select distinct rank from faculty ['What are the rank, first name, and last name of the faculty members?'
 'Which rank has the smallest number of faculty members?'
 'What are the faculty ids of all the male faculty members?'
 'What is the total number of faculty members?'
 'How many faculty members participate in an activity?'
 'What are the first and last name of the faculty who has the most students?'
 'Show all the ranks and the number of male and female faculty for each rank.'
 'Which building has most faculty members?']
 
4. select distinct rank from faculty ['For each faculty rank, show the number of faculty members who have it.'
 'How many faculty members do we have for each faculty rank?'
 'Find the faculty rank that has the least members.'
 'Show the rank, first name, and last name for all the faculty.'
 'How many faculty members do we have for each rank and gender?'
 'Show all the faculty ranks and the number of students advised by each rank.'
 'How many students are advised by each rank of faculty? List the rank and the number of students.']
 
5. select facid from faculty where sex = value ['Show the ids for all the faculty members who have at least 2 students.'
 'Show the ids of all the faculty members who participate in an activity and advise a student.'
 'Show the ids of students whose advisors are professors.'
 'Show the faculty id of each faculty member, along with the number of students he or she advises.'
 "Show ids for the faculty members who don't advise any student."
 'What are ids of the faculty members who not only participate in an activity but also advise a student.']
 
6. select count ( distinct facid ) from faculty_participates_in ['What are the faculty id and the number of students each faculty has?'
 'What is the first and last name of the faculty members who participated in at least one activity? For each of them, also show the number of activities they participated in.'
 'Show all the activity names and the number of faculty involved in each activity.'
 'Show the number of male and female assistant professors.']
 
7. select fname , lname from faculty where rank = value ['Show the first name, last name, and phone number for all female faculty members.'
 'Show the first and last name of all the faculty members who participated in some activity, together with the number of activities they participated in.'
 'Show first name and last name for all the students advised by Michael Goodrich.'
 'Give me the the first and last name of the faculty who advises the most students.'
 'Find the first and last name of the faculty who is involved in the largest number of activities.']
 
8. select stuid from student except select stuid from participates_in ['What are the ids of the students who are under 20 years old and are involved in at least one activity.'
 "Show the ids of the faculty who don't participate in any activity."
 'Show the ids for all the students who participate in an activity and are under 20.'
 'What are the ids of the faculty members who do not advise any student.']
 
9. select count ( * ) from faculty where rank = value and building = value ['How many female Professors do we have?'
 'How many male and female assistant professors do we have?'
 'Count the number of female Professors we have.'
 'Give me the number of faculty members who participate in an activity'
 'Show all the buildings that have at least 10 professors.'
 'What buildings have faculty offices?']
 
10. select fname , lname from faculty where rank = value ['What are the first name, last name, and phone number of all the female faculty members?'
 'What are the first names of the faculty members playing both Canoeing and Kayaking?'
 'What is the first and last name of the student participating in the most activities?']
 
11. select facid from faculty except select facid from faculty_participates_in ['Which faculty members advise two ore more students? Give me their faculty ids.'
 'What ranks do we have for faculty?'
 'Which students have professors as their advisors? Find their student ids.'
 'Which activity has the most faculty members participating in? Find the activity name.'
 'How many faculty members participate in each activity? Return the activity names and the number of faculty members.'
 'What are the ids of the students who are not involved in any activity'
 'What is the name of the activity that has the most faculty members involved in?'
 "Show the ids of the students who don't participate in any activity."
 'Find the first names of professors who are not playing Canoeing or Kayaking.'
 'Find the list of distinct ranks for faculty.']
 
12. select t2 . fname , t2 . lname from faculty as t1 join student as t2 on t1 . facid = t2 . advisor where t1 . fname = value and t1 . lname = value ['What are the first name and last name of all the instructors?'
 'Tell me the first and last name of the student who has the most activities.']
 
13. select distinct t1 . lname from faculty as t1 join faculty_participates_in as t2 on t1 . facid = t2 . facid join activity as t3 on t2 . actid = t2 . actid where t3 . activity_name = value or t3 . activity_name = value ['Find the ids of the students who participate in Canoeing and Kayaking.'
 'Show the first name and last name for all the instructors.'
 'Which students participate in both Canoeing and Kayaking as their activities? Tell me their student ids.']
